# Chapter 2 Root finding

## The Problem:

Find the root of an arbitrary function, i.e., find the $x$ that makes:
$$
f(x) = 0.
$$
Some remarks:
- All equations can be cast into the form above by moving all terms to the left
- The above can be understood as a vector equation, $n$ variables satisfying $n$ equations, which is **much harder**:
$$
\mathbf{f}(\mathbf{x})=\mathbf{0}.
$$
- Globally Convergent _vs_ Locally Convergent
- The success of solving the equation is _iterative_, and heavily depends on the "initial guess" -- there are many pitfalls.
- So, first, look at your function

## Bracketing and Bisection

General principle: if $f(a)$ and $f(b)$ have opposite signs and $f(x)$ is continuous, then there must be a root in $[a,b]$.
### Bisection

Suppose $f(a) < 0$ and $f(b) > 0$. Pick the mid-point $c = \frac{a+b}2$, if $f(c) = 0$ we are done. Else if $f(c) < 0$, the new intervel is $[c,b]$, otherwise $[a,c]$.

In [ ]:
def bisect(f, a, b, tolerance = 1.e-10):
    x = .5 * (a + b)
    fx = f(x)

    print(x, fx, b-a)

    if fx == 0. or b - a < tolerance:
        return x

    if f(a) * fx < 0.:
        y = bisect(f, a, x, tolerance)
    else:
        y = bisect(f, x, b, tolerance)

    return y

In [ ]:
from math import sin

def test_func(x):
    return x * sin(x) - 1.

print(bisect(test_func, 0., 2.))

#### Remark

Use lambda expression for function object:

In [ ]:
print(bisect(lambda x: x * sin(x) - 1, 0., 2.))

#### Remark

Do not evaluate function twice:

In [ ]:
def bisect_1(f, a, b, fa, fb, tolerance = 1.e-10):
    x = .5 * (a + b)
    fx = f(x)

    print(x, fx, b-a)

    if abs(fx) < tolerance or b - a < tolerance:
        return x

    if fa * fx < 0.:
        y = bisect_1(f, a, x, fa, fx, tolerance)
    else:
        y = bisect_1(f, x, b, fx, fb, tolerance)

    return y

In [ ]:
print(bisect_1(lambda x: x * sin(x) - 1, 0., 2., -1., 2. * sin(2.) - 1.))

In [ ]:
def bisect_2(f, a, b, tolerance = 1.e-10):

    fa = f(a)
    fb = f(b)
    while True:
        x = .5 * (a + b)
        fx = f(x)

        print(x, fx, b-a)

        if abs(fx) < tolerance or b - a <= tolerance:
            break

        if fa * fx < 0.:
            b = x
            fb = fx
        else:
            a = x
            fa = fx

    return x

In [ ]:
print(bisect_2(lambda x: x * sin(x) - 1, 0., 2.))

### Method of false position

The streight line between $(a, f(a))$ and $(b, f(b))$ is:
$$
y = f(a) + (x - a)\frac{f(b) - f(a)}{b-a}.
$$
It intgersects with the $x$-axis at:
$$
x_0 = a - \frac{f(a)(b-a)}{f(b) - f(a)}.
$$
Chose this point as the new point instead of the mid point.

In [ ]:
def false_position(f, a, b, tolerance = 1.e-10):

    fa = f(a)
    fb = f(b)
    assert(fa * fb < 0.)

    while True:
        x = a - fa * (b-a) / (fb - fa)
        fx = f(x)

        print(x, fx, b-a)

        if abs(fx) <= tolerance or b - a <= tolerance:
            break

        if fa * fx < 0.:
            b, fb = x, fx
        else:
            a, fa = x, fx

    return x

In [ ]:
print(false_position(lambda x: x * sin(x) - 1, 0., 2.))

### Pitfalls

1. multiple roots very close to each other
2. multiple roots
3. Shallow function

## Newton-Raphson method

### The fixed point of iterative methods

- Iteration for solving $x = g(x)$;
- Iteration: $p_{k+1} = g(p_k)$ is called **fixed-point iteration**;
- A fixed point of a function $g(x)$ is defined as the $x=P$ that makes $P = g(P)$;
- If the fixed-point iteration converges, it converges to the fixed-point;
  
  Let $\lim_{k\rightarrow\infty}p_k = p^\ast$. Take the limit of the iterative relation:
$$
p_{k+1} = g(p_k)
$$
and note that $g(x)$ is continuous:
$$
\lim_{k\rightarrow\infty}p_{k+1} = p^\ast = g(\lim_{k\rightarrow\infty}p_k) = g(p^\ast),
$$
therefore $p^\ast$ is a fixed point.

- Geometric interpretation;
- For the function $y = g(x)$ defined over $x\in[a,b]$, if $y\in[a,b]$, then $g$ has a fixed point in $[a,b]$.
- Furthermore, if $g'(x)$ is defined over $(a,b)$ and $\exists K < 1$ such that $|g'(x)|\leq K$ for all $x\in [(a,b)$, then the fixed point is unique.
- If $|g'(x)|\leq K < 1$ for all $x\in [a,b]$, then the iteration converges;
- If $|g'(x)| > 1$ for all $x\in [a,b]$, then the iteration diverges.
$$
|P-p_1| = |g(P) - g(p_0| = |g'(c)||P-p_0|
$$

### The Newton Raphson theorem

- Geometrically, approximate the function as a linear function using $f(x)$ and $f'(x)$.
- Algebraically:
$$
f(x)\cong f(x_0) + f'(x_0)(x-x_0),
$$
and solve $x$ from $f(x) = 0$:
$$
x = x_0 - \frac{f(x_0)}{f'(x_0)}.
$$
Use this as the next guess, _i.e._, $x_1$ and repeat the step above. The fixed point of:
$$
g(x) = x - \frac{f(x)}{f'(x)}.
$$
- Does the iteration converge?
$$
g'(x_0) = \frac{f(x_0)f''(x_0)}{(f'(x_0))^2}\quad\mbox{vanish if}\quad f(x_0) = 0.
$$
There is a vicinity of $x_0$ where $|g'(x)|<1$.
- Example: Newton Raphson for finding square root
$$
f(x) = x^2-A = 0
$$
  The iterative function:
$$
g(x) = x - \frac{x^2-A}{2x} = \frac 12\left(x+\frac Ax\right).
$$

## <span style="color:red">Exercises</span>
> - Rewrite Programs 2.2, 2.3, and 2.4 in python and do A&P 1 and 2 of Section 2.3
> - Section 2.4, Problem 11

In [ ]:
def newton_raphson_iteration(f, df, x0, accuracy):

    p = [x0]  # create a list to stores all iteration results
    while True:
        x = x0 - f(x0) / df(x0)

        p.append(x)

        if np.abs(x-x0) < accuracy:
            break

        x0 = x

    return p

A test function with a simple root at $x = -2$ and a double root at $x = 1$:

In [ ]:
def test_func(x):
    return x ** 3 - 3 * x + 2.

def d_test_func(x):
    return 3 * x ** 2 - 3

#
# How does the function look like?
#

import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(-2.5, 2.5)

fx = test_func(x)
dx = d_test_func(x)

plt.grid(True)
plt.plot(x, fx)
plt.plot(x, dx)

plt.show()

In [ ]:
newton_raphson_iteration(test_func, d_test_func, -2.5, 1.e-10)

### Convergence Speed
- How to measure the speed?  Define the order of convergence. If
$$
\lim_{n\rightarrow\infty}\frac{|E_{n+1}|}{|E_n|^R} =
\lim_{n\rightarrow\infty}\frac{|p-p_{n+1}|}{|p-p_n|^R} = A,
$$
$R$ is called the **order of convergence**.

In [ ]:
p1 = newton_raphson_iteration(test_func, d_test_func, -2.4, 1.e-10)
p2 = newton_raphson_iteration(test_func, d_test_func, 1.4, 1.e-10)

e1 = np.abs(np.array(p1)+2.)
e2 = np.abs(np.array(p2)-1.)

plt.grid(True)
plt.semilogy(e1, 'ro', label='$x_0 = -2$')
plt.semilogy(e2, 'bo', label='$x_0 = 1$')

plt.xlabel("Step")
plt.ylabel("Error")
plt.legend()
plt.show()

- Convergence Rate:
$$
\begin{eqnarray}
\frac{|E_{n+1}|}{|E_{n}|^2} &\cong& \frac{f''(x_0)}{2|f'(x_0)|}\quad\mbox{Simple root}\\
\frac{|E_{n+1}|}{|E_{n}|} &\cong& \frac{M-1}{M}\quad\mbox{Multiple root}\\
\end{eqnarray}
$$

## Accelerated Newton-Raphson
- If the order of the root is known, Newton-Raphson can be accelertated to achieve quadratic convergence.
$$
x_1 = x_0 - \frac{Mf(x_0)}{f'(x_0)}.
$$

In [ ]:
def accelerated_newton_raphson_iteration(f, df, x0, accuracy, m=1.):

    p = [x0]  # create a list to stores all iteration results
    while True:
        x = x0 - m * f(x0) / df(x0) 

        p.append(x)

        if np.abs(x-x0) < accuracy:
            break

        x0 = x

    return p

In [ ]:
p1 = accelerated_newton_raphson_iteration(test_func, d_test_func,-2.4, 1.e-10)
p2 = accelerated_newton_raphson_iteration(test_func, d_test_func, 1.4, 1.e-10)
p3 = accelerated_newton_raphson_iteration(test_func, d_test_func, 1.4, 1.e-10, 2)

e1 = np.abs(np.array(p1)+2.)
e2 = np.abs(np.array(p2)-1.)
e3 = np.abs(np.array(p3)-1.)

plt.semilogy(e1, 'ro', label='$x_0 = -2$')
plt.semilogy(e2, 'bo', label='$x_0 = 1, m = 1$')
plt.semilogy(e3, 'go', label='$x_0 = 1, m = 2$')

plt.grid(True)
plt.xlabel("Step")
plt.ylabel("Error")
plt.legend()
plt.show()

## Secant Method

- If the derivative is _not_ known, replace the tengent line with the secant line.
$$
f'(x_k) \cong \frac{f(x_k)-f(x_{k-1})}{x_k - x_{k-1}}
$$
The iteration:
$$
x_2 = x_1 - \frac{f(x_1)(x_1 - x_0)}{f(x_1) - f(x_0)}.
$$
- One function evaluation

In [ ]:
def secant_iteration(f, x0, x1, accuracy):

    p = [x0, x1]  # create a list to stores all iteration results
    fx0 = f(x0)

    while True:
        fx1 = f(x1)
        x = x1 - f(x1) * (x1 - x0) / (fx1 - fx0)

        p.append(x)

        if np.abs(x-x1) < accuracy:
            break

        x0 = x1
        x1 = x
        fx0 = fx1
        
    return p

In [ ]:
p1 = newton_raphson_iteration(test_func, d_test_func, -1.4, 1.e-11)
p2 = secant_iteration(test_func, -1.4, -1.2, 1.e-11)

e1=np.abs(np.array(p1)+2.)
e2=np.abs(np.array(p2)+2.)

plt.grid(True)
plt.semilogy(e1, 'ro', label='Newton-Raphson')
plt.semilogy(e2, 'bo', label='Secant')

plt.legend()
plt.show()

## <span style="color:red">Exercises</span>
> - Section 2.4, Exercise 18
> - Section 2.4, Exercise 22, write a python program implementing the algorithm

## Aitken's Process

Accelerating _any_ linearly-convergent sequence.

By definition, if $p_n$ linearly converge to $p$ as $n\rightarrow\infty$, then:
$$
\frac{p-p_{n+1}}{p-p_n} \cong A.
$$
For two consecutive steps, we have:
$$
\frac{p-p_{n+1}}{p-p_n} \cong \frac{p-p_{n+2}}{p-p_{n+1}},
$$
from which we can solve $p$:
$$
p\cong\frac{p_{n+2}p_n - p^2_{n+1}}{p_{n+2}-2p_{n+1}+p_n}=
p_n-\frac{(p_{n+1}-p_n)^2}{(p_{n+2}-p_{n+1})-(p_{n+1}-p_n)}
=p_n-\frac{(\Delta p_n)^2}{\Delta^2p_n}.
$$

In [ ]:
def steffensen_iteration(f, df, p0, accuracy):

    p = [p0]  # create a list to store all iteration results
    i = 0
    q = [ ]
    while True:
        i += 1

        p1 = p0 - f(p0) / df(p0)
        p2 = p1 - f(p1) / df(p1)
        q0 = p0 - ((p1-p0)**2)/(p2-2*p1+p0)
         
        p.append( p0 )
        q.append( q0 )
        if abs(q0-p2) < accuracy:
            break

        p0 = p2
        #p1 = p2

    return p, q

In [ ]:
p, q = steffensen_iteration(test_func, d_test_func, 1.4, 1.e-11)

ep=np.abs(np.array(p)-1.)
eq=np.abs(np.array(q)-1.)

plt.grid(True)
plt.semilogy(ep, 'ro', label='p')
plt.semilogy(eq, 'bo', label='q')

plt.legend()
plt.show()

In [ ]:
def steffensen_iteration1(f, df, p0, accuracy):

    p = [p0]  # create a list to store all iteration results
    i = 0
    q = [ ]
    while True:
        i += 1

        p.append(p[-1] - f(p[-1]) / df(p[-1]))

        if i > 1:
            q.append(p[-3] - ((p[-2]-p[-3]) ** 2)/(p[-1]- 2*p[-2] + p[-3]))

            if i> 2 and abs(q[-1]-q[-2]) < accuracy:
                break

    return p, q

In [ ]:
p, q = steffensen_iteration1(test_func, d_test_func, 1.4, 1.e-10)

ep=np.abs(np.array(p)-1.)
eq=np.abs(np.array(q)-1.)

plt.grid(True)
plt.semilogy(ep, 'ro', label='p')
plt.semilogy(eq, 'bo', label='q')

plt.legend()
plt.show()

## <span style="color:red">Exercises</span>
> Section 2.5, Exercise 10, 14